<a href="https://colab.research.google.com/github/yusuku/DepthEstimationFrom360/blob/main/SimplePytorchOnnxModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install onnxruntime
!pip install onnx


In [15]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import onnxruntime as ort



In [16]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [17]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
model = NeuralNetwork().to(device)

Using cpu device


In [18]:
torch_input=torch.randn((1,1,28,28))
torch_output=model(torch_input)

In [19]:
torch_output

tensor([[0.0000, 0.0000, 0.0346, 0.0726, 0.0000, 0.1017, 0.1527, 0.0812, 0.0627,
         0.0326]], grad_fn=<ReluBackward0>)

In [20]:
torch_output.shape

torch.Size([1, 10])

In [21]:
model.eval()
torch.onnx.export(model,                     # model being run
                  torch_input,               # model input (or a tuple for multiple inputs)
                  "sup.onnx",                # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=15,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

In [22]:
torch_input=torch.ones((1,1,28,28))
torch_output=model(torch_input)
ort_sess = ort.InferenceSession('/content/sup.onnx')
onnx_outputs = ort_sess.run(None, {'input': torch_input.numpy()})


In [23]:
onnx_outputs

[array([[0.        , 0.08978456, 0.        , 0.0351091 , 0.07517748,
         0.0515485 , 0.08128192, 0.16308148, 0.02992797, 0.        ]],
       dtype=float32)]